In [1]:
import pandas as pd
import numpy as np
from glob import glob
import os
from pathlib import Path
from datetime import date
import json
import time
from config import *
from send_mail import send_report
from itertools import product
from datetime import datetime, timedelta
xlsx_dir_path = r"\\bosch.com\dfsRB\DfsCN\loc\WX4\Dept\TER\10_QMM\13 QMC\14 Reporting\00 Monthly quality reports\PowerBi\Customer 2023\Able to clean customers\Final_Output_folder"

In [ ]:
Customer_summary_Output_By_FailMonth = pd.read_excel(os.path.join(xlsx_dir_path,"Customer_summary_Output_By_FailMonth.xlsx"))
Customer_summary_Output_By_FailMonth = Customer_summary_Output_By_FailMonth[Customer_summary_Output_By_FailMonth.Fail_Month.notna()]
Customer_summary_Output_By_FailMonth.loc[:,"Totol_Cost_Sum"] = Customer_summary_Output_By_FailMonth.loc[:,"Totol_Cost_Sum"].fillna(0)
Customer_summary_Output_By_FailMonth = Customer_summary_Output_By_FailMonth[Customer_summary_Output_By_FailMonth.loc[:,"Material"].apply(lambda x:len(str(x))==12)]  # EXCLUDE ["非RBCW产品",NAN]
Customer_summary_Output_By_FailMonth.loc[:,"year"] = Customer_summary_Output_By_FailMonth.loc[:,"Fail_Month"].apply(lambda x:int(str(x)[:4]))
Customer_summary_Output_By_FailMonth.to_excel(os.path.join(xlsx_dir_path,"Customer_summary_Output_By_FailMonth_detail.xlsx"))
Customer_summary_Output_By_FailYear = Customer_summary_Output_By_FailMonth.dropna(subset="TNS_Year")
Customer_summary_Output_By_FailYear = Customer_summary_Output_By_FailYear.drop_duplicates(subset=["Customer_Name", "Material","year"])
Customer_summary_Output_By_FailYear = Customer_summary_Output_By_FailYear.loc[:,["Customer_Name", "year", "TNS_Year"]]
Customer_summary_Output_By_FailYear = Customer_summary_Output_By_FailYear.groupby(by=["Customer_Name","year"]).sum()
Customer_summary_Output_By_FailMonth_year_cost = Customer_summary_Output_By_FailMonth[["Customer_Name","Totol_Cost_Sum","year"]].groupby(by=["Customer_Name","year"]).sum()
Customer_summary_Output_By_FailMonth_year_edc = Customer_summary_Output_By_FailMonth_year_cost.merge(Customer_summary_Output_By_FailYear, left_index=True,right_index=True,how="right")
# Customer_summary_Output_By_FailMonth_year_edc.loc[:,"TNS"] = Customer_summary_Output_By_FailMonth_year_edc.loc[:,"TNS"].apply(lambda x:np.abs(x))
Customer_summary_Output_By_FailMonth_year_edc.loc[:,"edc"] = Customer_summary_Output_By_FailMonth_year_edc.loc[:,"Totol_Cost_Sum"]/ Customer_summary_Output_By_FailMonth_year_edc.loc[:,"TNS_Year"]
Customer_summary_Output_By_FailMonth_year_edc.loc[:,"current_year"] = date.today().year
Customer_summary_Output_By_FailMonth_year_edc.loc[:,"current_month"] = date.today().month
def filter_month(current_month : int):
    if date.today().day > 20:
        return current_month
    else:
        return current_month - 1

Customer_summary_Output_By_FailMonth_year_edc.loc[:,"current_month"] = Customer_summary_Output_By_FailMonth_year_edc.loc[:,"current_month"].apply(lambda x:filter_month(x))
Customer_summary_Output_By_FailMonth_year_edc = Customer_summary_Output_By_FailMonth_year_edc[Customer_summary_Output_By_FailMonth_year_edc.index.get_level_values("year") == Customer_summary_Output_By_FailMonth_year_edc.current_year]
Customer_summary_Output_By_FailMonth_year_edc = Customer_summary_Output_By_FailMonth_year_edc[Customer_summary_Output_By_FailMonth_year_edc.edc > EDC]
Customer_summary_Output_By_FailMonth_year_edc.loc[:,"edc"] = Customer_summary_Output_By_FailMonth_year_edc.loc[:,"edc"].apply(lambda x:float(x * 100))
Customer_summary_Output_By_FailMonth_year_edc.loc[:,"edc"] = Customer_summary_Output_By_FailMonth_year_edc.loc[:,"edc"].apply(lambda x:np.round(x,2).astype(str) + ' %')

Customer_summary_Output_By_FailMonth_year_edc.to_excel(os.path.join(xlsx_dir_path,"Customer_summary_Output_By_FailMonth_year_edc.xlsx"))
Customer_summary_Output_By_FailMonth_year_edc = Customer_summary_Output_By_FailMonth_year_edc.reset_index()

for customer in Customer_summary_Output_By_FailMonth_year_edc.Customer_Name.unique():
    customer_df = Customer_summary_Output_By_FailMonth_year_edc[Customer_summary_Output_By_FailMonth_year_edc.Customer_Name == customer]
    report_text = "\n"
    report_list= []
    for json_data in json.loads(customer_df.to_json(orient='records')):
        Customer_Name = json_data["Customer_Name"]
        # Product_Name = json_data["Product_Name"]
        current_year = json_data["year"]
        current_month = json_data["current_month"]
        break
    report_tuple = tuple(report_list)
    report_text = report_text.join(report_tuple)
    df2 = Customer_summary_Output_By_FailMonth[Customer_summary_Output_By_FailMonth.Customer_Name == customer]
    send_report(Subject="EDC Warning",content_1=f"您好， {Customer_Name}客户,{current_year}年{current_month}月YTD, 已超过公司质量目标，请调查原因及制定改善措施，谢谢。",df=customer_df, df2=df2, to_all=True)
    time.sleep(10)
Customer_summary_Output_By_FailMonth_year_edc


In [ ]:
Customer_summary_Output_By_FailMonth_year_edc

In [ ]:
def filter_month(current_month : int):
    if date.today().day > 20:
        return current_month
    else:
        return current_month - 1
Customer_summary_Output_By_FailMonth = pd.read_excel(os.path.join(xlsx_dir_path,"Customer_summary_Output.xlsx"))
Customer_summary_Output_By_FailMonth = Customer_summary_Output_By_FailMonth[Customer_summary_Output_By_FailMonth.Mileage <= 1000]
Customer_summary_Output_By_FailMonth['year'] = pd.to_datetime(Customer_summary_Output_By_FailMonth['Statistic month'], errors='coerce').dt.year
Customer_summary_Output_By_FailMonth['month'] = pd.to_datetime(Customer_summary_Output_By_FailMonth['Statistic month'], errors='coerce').dt.month
row_df = Customer_summary_Output_By_FailMonth[Customer_summary_Output_By_FailMonth.year == date.today().year]
row_df = row_df[Customer_summary_Output_By_FailMonth.month == filter_month(date.today().month)]
Customer_summary_Output_By_FailMonth = Customer_summary_Output_By_FailMonth[["Customer", "Statistic month","Mileage"]].groupby(["Customer", "Statistic month"]).count()
Customer_summary_Output_By_FailMonth = Customer_summary_Output_By_FailMonth.rename(columns={"Mileage":"Mil<1000 Qty"})
Customer_summary_Output_By_FailMonth = Customer_summary_Output_By_FailMonth.reset_index()

Customer_summary_Output_By_FailMonth.loc[:,"year"] = Customer_summary_Output_By_FailMonth.loc[:,"Statistic month"].apply(lambda x:int(str(x)[:4]))
Customer_summary_Output_By_FailMonth['year'] = pd.to_datetime(Customer_summary_Output_By_FailMonth['Statistic month'], errors='coerce').dt.year
Customer_summary_Output_By_FailMonth['month'] = pd.to_datetime(Customer_summary_Output_By_FailMonth['Statistic month'], errors='coerce').dt.month

Customer_summary_Output_By_FailMonth_year_complaint = Customer_summary_Output_By_FailMonth
Customer_summary_Output_By_FailMonth_year_complaint.loc[:,"current_year"] = date.today().year
Customer_summary_Output_By_FailMonth_year_complaint.loc[:,"current_month"] = date.today().month


Customer_summary_Output_By_FailMonth_year_complaint.loc[:,"current_month"] = Customer_summary_Output_By_FailMonth_year_complaint.loc[:,"current_month"].apply(lambda x:filter_month(x))
Customer_summary_Output_By_FailMonth_year_complaint = Customer_summary_Output_By_FailMonth_year_complaint[Customer_summary_Output_By_FailMonth_year_complaint.year == Customer_summary_Output_By_FailMonth_year_complaint.current_year]
Customer_summary_Output_By_FailMonth_year_complaint = Customer_summary_Output_By_FailMonth_year_complaint[Customer_summary_Output_By_FailMonth_year_complaint.month == Customer_summary_Output_By_FailMonth_year_complaint.current_month]
Customer_summary_Output_By_FailMonth_year_complaint.to_excel(os.path.join(xlsx_dir_path,"Customer_summary_Output_By_FailMonth_year_complaint.xlsx"))

Customer_summary_Output_By_FailMonth_year_complaint = Customer_summary_Output_By_FailMonth_year_complaint[Customer_summary_Output_By_FailMonth_year_complaint.loc[:,"Mil<1000 Qty"] > 2]
Customer_summary_Output_By_FailMonth_year_complaint = Customer_summary_Output_By_FailMonth_year_complaint.reset_index(drop=True).sort_values("Customer")


for customer in Customer_summary_Output_By_FailMonth_year_complaint.Customer.unique():
    customer_df = Customer_summary_Output_By_FailMonth_year_complaint[Customer_summary_Output_By_FailMonth_year_complaint.Customer == customer]
    report_text = "\n"
    report_list= []
    for json_data in json.loads(customer_df.to_json(orient='records')):
        Customer_Name = json_data["Customer"]
        # Product_Name = json_data["Product_Name"]
        current_year = json_data["year"]
        current_month = json_data["month"]
        failure_quantity =json_data["Mil<1000 Qty"]
        break
    report_list.append(f"{Customer_Name}客户 {current_year}年{current_month}月 1000KM失效={failure_quantity}")
    report_tuple = tuple(report_list)
    report_text = report_text.join(report_tuple)
    df2 = row_df[row_df.Customer == customer]
    send_report(Subject="1000KM Complaint Warning",content_1=f"您好:\n {report_text}, 已超过公司质量目标，请调查原因及制定改善措施，谢谢。", df=customer_df,df2=df2,to_all=False)
    time.sleep(10)

C:\Users\bih1wx\AppData\Local\Temp\ipykernel_200\1616074837.py:11: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  row_df = row_df[Customer_summary_Output_By_FailMonth.month == filter_month(date.today().month)]
send: 'ehlo SZHM-C-0017Y.wx.apac.bosch.com\r\n'
reply: b'250-FE-MBX2075.de.bosch.com Hello [10.35.29.9]\r\n'
reply: b'250-SIZE 37748736\r\n'
reply: b'250-PIPELINING\r\n'
reply: b'250-DSN\r\n'
reply: b'250-ENHANCEDSTATUSCODES\r\n'
reply: b'250-STARTTLS\r\n'
reply: b'250-X-ANONYMOUSTLS\r\n'
reply: b'250-AUTH NTLM\r\n'
reply: b'250-X-EXPS GSSAPI NTLM\r\n'
reply: b'250-8BITMIME\r\n'
reply: b'250-BINARYMIME\r\n'
reply: b'250-CHUNKING\r\n'
reply: b'250 XRDST\r\n'
reply: retcode (250); Msg: b'FE-MBX2075.de.bosch.com Hello [10.35.29.9]\nSIZE 37748736\nPIPELINING\nDSN\nENHANCEDSTATUSCODES\nSTARTTLS\nX-ANONYMOUSTLS\nAUTH NTLM\nX-EXPS GSSAPI NTLM\n8BITMIME\nBINARYMIME\nCHUNKING\nXRDST'
send: 'STARTTLS\r\n'
reply: b'220 2.0.0 SMTP server ready\r\n'
reply: retco

In [1]:
Customer_summary_Output_By_FailMonth_year_complaint

NameError: name 'Customer_summary_Output_By_FailMonth_year_complaint' is not defined

In [ ]:
Customer_summary_Output_By_FailMonth = pd.read_excel(os.path.join(xlsx_dir_path,"Customer_summary_Output_By_FailMonth.xlsx"))
Customer_summary_Output_By_FailMonth = Customer_summary_Output_By_FailMonth[Customer_summary_Output_By_FailMonth.Fail_Month.notna()]
Customer_summary_Output_By_FailMonth = Customer_summary_Output_By_FailMonth[Customer_summary_Output_By_FailMonth.TNS_QTY.notna()]
Customer_summary_Output_By_FailMonth = Customer_summary_Output_By_FailMonth[Customer_summary_Output_By_FailMonth.Totol_Cost_Sum.notna()]
Customer_summary_Output_By_FailMonth.loc[:,"year"] = Customer_summary_Output_By_FailMonth.loc[:,"Fail_Month"].apply(lambda x:int(str(x)[:4]))
Customer_summary_Output_By_FailMonth.loc[:,"month"] = Customer_summary_Output_By_FailMonth.loc[:,"Fail_Month"].apply(lambda x:int(str(x)[5:7]))
Customer_summary_Output_By_FailMonth_exchange = Customer_summary_Output_By_FailMonth[["Customer_Name","Product_Name","Fail QYT","year","month"]].groupby(by=["Customer_Name","Product_Name","year","month"]).sum()
Customer_summary_Output_By_FailMonth_exchange.loc[:,"current_year"] = date.today().year
Customer_summary_Output_By_FailMonth_exchange.loc[:,"current_month"] = date.today().month
Customer_summary_Output_By_FailMonth_exchange = Customer_summary_Output_By_FailMonth_exchange[(Customer_summary_Output_By_FailMonth_exchange.index.get_level_values("year") == Customer_summary_Output_By_FailMonth_exchange.current_year) | (Customer_summary_Output_By_FailMonth_exchange.index.get_level_values("year") == Customer_summary_Output_By_FailMonth_exchange.current_year - 1)]

Customer_summary_Output_By_FailMonth_exchange = Customer_summary_Output_By_FailMonth_exchange.reset_index().sort_values("Customer_Name")

Customer_Name = list(Customer_summary_Output_By_FailMonth_exchange.Customer_Name.unique())
Product_Name = list(Customer_summary_Output_By_FailMonth_exchange.Product_Name.unique())
year = list(Customer_summary_Output_By_FailMonth_exchange.year.unique())
month = [x for x in range(1,13)]
master_df = pd.DataFrame(list(product(Customer_Name, Product_Name, year, month)), columns=["Customer_Name", "Product_Name", "year", "month"])
Customer_summary_Output_By_FailMonth_exchange = Customer_summary_Output_By_FailMonth_exchange.merge(master_df,on=["Customer_Name", "Product_Name", "year", "month"],how="right")
Customer_summary_Output_By_FailMonth_exchange = Customer_summary_Output_By_FailMonth_exchange.sort_values(["Customer_Name","Product_Name","year","month"])
Customer_summary_Output_By_FailMonth_exchange.loc[:,"Fail QYT"] = Customer_summary_Output_By_FailMonth_exchange.loc[:,"Fail QYT"].fillna(0)
Customer_summary_Output_By_FailMonth_exchange = Customer_summary_Output_By_FailMonth_exchange.ffill()
Customer_summary_Output_By_FailMonth_exchange = Customer_summary_Output_By_FailMonth_exchange.bfill()
Customer_summary_Output_By_FailMonth_exchange.loc[:,"incremental"] = Customer_summary_Output_By_FailMonth_exchange.groupby(["Customer_Name","Product_Name"])["Fail QYT"].diff()

def filter_month(current_month : int):
    if date.today().day > 20:
        return [current_month - 1, current_month]
    else:
        return [current_month - 2, current_month - 1]

Customer_summary_Output_By_FailMonth_exchange.loc[:,"current_month"] = Customer_summary_Output_By_FailMonth_exchange.loc[:,"current_month"].apply(lambda x:filter_month(x))
Customer_summary_Output_By_FailMonth_exchange_2month = Customer_summary_Output_By_FailMonth_exchange[Customer_summary_Output_By_FailMonth_exchange.month.isin(Customer_summary_Output_By_FailMonth_exchange.loc[:,"current_month"].drop_duplicates().values[0])]
Customer_summary_Output_By_FailMonth_exchange_2month = Customer_summary_Output_By_FailMonth_exchange_2month.groupby(["Customer_Name","year","Product_Name"]).min()[["incremental"]]
Customer_summary_Output_By_FailMonth_exchange_2month = Customer_summary_Output_By_FailMonth_exchange_2month[Customer_summary_Output_By_FailMonth_exchange_2month.index.get_level_values("year") == date.today().year]
Customer_summary_Output_By_FailMonth_exchange_2month.loc[:,"month"] = max(Customer_summary_Output_By_FailMonth_exchange.loc[:,"current_month"].drop_duplicates().values[0])
Customer_summary_Output_By_FailMonth_exchange_2month.to_excel("C:\disk\AI\ATS_EDC_EWS\data_for_power_bi\Customer_summary_Output_By_FailMonth_exchange_2month.xlsx")
Customer_summary_Output_By_FailMonth_exchange_2month = Customer_summary_Output_By_FailMonth_exchange_2month[Customer_summary_Output_By_FailMonth_exchange_2month.incremental > 0]
Customer_summary_Output_By_FailMonth_exchange_2month = Customer_summary_Output_By_FailMonth_exchange_2month.reset_index().sort_values("Customer_Name")

Customer_summary_Output_By_FailMonth_exchange_12month = Customer_summary_Output_By_FailMonth_exchange
Customer_summary_Output_By_FailMonth_exchange_12month.loc[:,"current_month"] = max(Customer_summary_Output_By_FailMonth_exchange.loc[:,"current_month"].drop_duplicates().values[0])
df = Customer_summary_Output_By_FailMonth_exchange_12month
# Assume df is your DataFrame
df['date'] = pd.to_datetime(df['year'].astype(int).astype(str) + '-' + df['month'].astype(int).astype(str) + '-01')
df['current_date'] = pd.to_datetime(df['current_year'].astype(int).astype(str) + '-' + df['current_month'].astype(int).astype(str) + '-01')

# Filter rows where 'date' is within 12 months before 'current_date'
mask = df['date'] > df['current_date'] - timedelta(days=365)
df_12m = df[mask].copy()

# Group by keys and compute average of previous 12 months
avg_df = (
    df_12m
    .groupby(['Customer_Name', 'Product_Name', 'current_year', 'current_month'])['Fail QYT']
    .mean()
    .reset_index()
    .rename(columns={'Fail QYT': 'last_12_month'})
)

# Keep only rows that match current_year and current_month
df_current = df[
    (df['year'] == df['current_year']) & 
    (df['month'] == df['current_month'])
].copy()

# Merge the calculated average into current-month rows
final_df = pd.merge(df_current, avg_df, on=['Customer_Name', 'Product_Name', 'current_year', 'current_month'], how='left')

# Drop helper columns if not needed
final_df.drop(columns=['date', 'current_date'], errors='ignore', inplace=True)

# Final result
final_df
final_df.to_excel("C:\disk\AI\ATS_EDC_EWS\data_for_power_bi\Customer_summary_Output_By_FailMonth_exchange_12month.xlsx")
Customer_summary_Output_By_FailMonth_exchange_12month = final_df[final_df.last_12_month < final_df.loc[:,"Fail QYT"]]


In [ ]:
for customer in Customer_summary_Output_By_FailMonth_exchange_2month.Customer_Name.unique():
    customer_df = Customer_summary_Output_By_FailMonth_exchange_2month[Customer_summary_Output_By_FailMonth_exchange_2month.Customer_Name == customer].reset_index(drop=True)
    report_text = "\n"
    report_list= []
    for json_data in json.loads(customer_df.to_json(orient='records')):
        Customer_Name = json_data["Customer_Name"]
        Product_Name = json_data["Product_Name"]
        current_year = json_data["year"]
        current_month = json_data["month"]
        break
    report_list.append(f"{Customer_Name}客户 {current_year}年{current_month}月 连续两个月换件数量增加")
    report_tuple = tuple(report_list)
    report_text = report_text.join(report_tuple)
    send_report(Subject="换件数量监控",content_1=f"您好:\n {report_text}, 已超过公司质量目标，请调查原因及制定改善措施，谢谢。", df=customer_df,to_all=True)
    time.sleep(10)

In [ ]:
for customer in Customer_summary_Output_By_FailMonth_exchange_12month.Customer_Name.unique():
    customer_df = Customer_summary_Output_By_FailMonth_exchange_12month[Customer_summary_Output_By_FailMonth_exchange_12month.Customer_Name == customer]
    report_text = "\n"
    report_list= []
    for json_data in json.loads(customer_df.to_json(orient='records')):
        Customer_Name = json_data["Customer_Name"]
        Product_Name = json_data["Product_Name"]
        current_year = json_data["year"]
        current_month = json_data["month"]
        break
    report_list.append(f"{Customer_Name}客户 {Product_Name}产品 {current_year}年{current_month}月 换件数量超过过去12月平均值")
    report_tuple = tuple(report_list)
    report_text = report_text.join(report_tuple)
    send_report(Subject="换件数量监控",content_1=f"您好:\n {report_text}, 已超过公司质量目标，请调查原因及制定改善措施，谢谢。",df=customer_df, to_all=True)


In [ ]:
df.dtypes

In [ ]:
df = Customer_summary_Output_By_FailMonth_exchange_12month[Customer_summary_Output_By_FailMonth_exchange_12month.month == 6].sort_values("Fail QYT")
df[df.year == 2025]

In [ ]:
# Customer_summary_Output_By_FailMonth_exchange
# def filter_last_12month(df, x):
#     temp_df = df[df]

Customer_summary_Output_By_FailMonth_exchange_12month = Customer_summary_Output_By_FailMonth_exchange[Customer_summary_Output_By_FailMonth_exchange.month == max((Customer_summary_Output_By_FailMonth_exchange.loc[:,"current_month"].drop_duplicates().values[0]))]

In [ ]:
Customer_summary_Output_By_FailMonth_exchange.month.isin(Customer_summary_Output_By_FailMonth_exchange.loc[:,"current_month"].drop_duplicates().values[0]).value_counts()

In [ ]:
Customer_summary_Output_By_FailMonth_exchange[(Customer_summary_Output_By_FailMonth_exchange.Customer_Name == "JMC") & (Customer_summary_Output_By_FailMonth_exchange.year == 2025)][-20:]

In [ ]:
report_text

In [ ]:
json.loads(Customer_summary_Output_By_FailMonth_year_complaint.to_json(orient='records'))

In [ ]:
Customer_summary_Output_By_FailMonth

In [ ]:
Customer_summary_Output_By_FailMonth_year_complaint.reset_index().sort_values("Customer_Name")